In [1]:
from pathlib import Path
from tabulate import tabulate
from IPython.display import display,Markdown,HTML
import re
import os
import anthropic
from groq import Groq
from openai import OpenAI
import ollama

In [2]:
# util functions
def format_xml(xml_string):
    paragraphs = xml_string.split('\n\n')
    wrapped_paragraphs = [textwrap.fill(p, 72) for p in paragraphs]
    wrapped_string = '\n\n'.join(wrapped_paragraphs)
    soup = BeautifulSoup(wrapped_string, 'html.parser')
    pretty_xml = soup.prettify()
    colored_xml = pretty_xml.replace('<', Fore.RED + '<').replace('>', '>' + Fore.RESET)
    print(colored_xml)
    
def split_into_sentences(text):
    sentences = nltk.tokenize.sent_tokenize(text, language='german')
    return sentences

In [3]:
import json
from pathlib import Path
from jinja2 import Environment, FileSystemLoader, meta

class JinManager:
    def __init__(self, dir_path):
        self.dir = Path(dir_path)
        self.env = Environment(loader=FileSystemLoader(str(self.dir)))
        self.schema = self._load_tmpls()

    def _load_tmpls(self):
        templates = {}
        for file in self.dir.glob('*.j2'):
            tmpl = self.env.get_template(str(file.name))
            source = self.env.loader.get_source(self.env, str(file.name))[0]
            parsed_content = self.env.parse(source)
            templates[file.stem] = {
                'tmpl': tmpl,
                'params': list(meta.find_undeclared_variables(parsed_content)),
                'related': []
            }
        return templates
    
    def list(self):
        return list(self.schema.keys())

    def render(self, tmpl_name, params):
        if tmpl_name not in self.schema:
            raise ValueError(f"No template named '{tmpl_name}' found.")
        template = self.schema[tmpl_name]['tmpl']
        return template.render(params)

In [4]:
root = "/Users/friedrichstr/Dropbox/0Audio"
files = list(Path(root).glob("*.qmd"))
ai_vorl = {f.stem: f.read_text() for f in files}
files = [[name, len(text)] for name, text in ai_vorl.items()]
files = sorted(files, key=lambda x: x[0])
table = tabulate(files, headers=['f.stem', 'len(value)'], tablefmt='html')
table

f.stem,len(value)
ai_Vorl1,67977
ai_Vorl2,67715
ai_Vorl3,64550
ai_Vorl4,66323


In [5]:
sys_mess = '''
Aufgabe ist, den folgenden Text in lesbarer Formulierung für einen erweiterten, umfassenden Artikel umzuschreiben.
Leser sind akademische Experten, die sich im Bereich KI und Philosophie auskennen.
Ausgabe soll nur der Text sein, keine weiteren Informationen. Bleiben Sie in der deutschen Sprache.
'''

In [6]:
def instruct_execute(sys_mess, instruct, model_short):
    model_vendors = {
        'claude-opus': {"model":"claude-3-opus-20240229",'vendor': 'anthropic', 'api_key': 'ANTHROPIC_API_KEY'},
        'claude-haiku': {'model': 'claude-3-haiku-20240307','vendor': 'anthropic', 'api_key': 'ANTHROPIC_API_KEY'},
        'groq-llama3': {'model':'llama3-70b-8192','vendor': 'groq', 'api_key': 'GROQ_API_KEY'},
        'groq-mixtral': {'model':'mixtral-8x7b-32768','vendor': 'groq', 'api_key': 'GROQ_API_KEY'},
        'gpt-4o': {'model':'gpt-4o','vendor': 'openai', 'api_key': 'OPENAI_API_KEY'},
        'dbrx': {'model':'dbrx','vendor': 'ollama', 'api_key': ''},
        'llama3': {'model':'llama3','vendor': 'ollama', 'api_key': ''},
        'llama3-70': {'model':'llama3:70b','vendor': 'ollama', 'api_key': ''},
        'mixtral': {'model':'mixtral:8x22b','vendor':'ollama','api_key': ''},
        'llama3-chatqa': {'model':'llama3-chatqa:70b-v1.5-q4_1','vendor': 'ollama', 'api_key': ''},
    }

    if model_short not in model_vendors:
        raise ValueError("Invalid model")

    model = model_vendors[model_short]['model']
    vendor = model_vendors[model_short]['vendor']
    api_key = os.getenv(model_vendors[model_short]['api_key'])

    print(f"Chosen model: {model}, Vendor: {vendor.capitalize()}")

    if vendor == 'anthropic':
        client = anthropic.Anthropic(api_key=api_key)
        message = client.messages.create(
            model=model, 
            max_tokens=4096,
            system=sys_mess,
            messages=[{"role": "user", "content": [{"type": "text", "text": instruct}]}]
        )
        gg = message.content[0].text

    elif vendor == 'groq':
        client = Groq(api_key=api_key)
        message = client.chat.completions.create(
            messages=[
                {"role": "system", "content":sys_mess},
                {"role": "user", "content": instruct}
            ],
            model=model,
            temperature=0.0,
            response_format={"type": "text"},
        )
        gg = message.choices[0].message.content

    elif vendor == 'openai':
        client = OpenAI(api_key=api_key)
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": sys_mess},
                {"role": "user", "content": instruct}
            ]
        )
        gg = response.choices[0].message.content

    elif vendor == 'ollama':
        response = ollama.chat(
            model=model, 
            messages=[
                {'role': 'system', 'content': sys_mess},
                {'role': 'user', 'content': instruct},
            ]
        )
        gg = response['message']['content']

    return gg, vendor



In [7]:
templ_dir='/Users/friedrichstr/Dropbox/202x/2024projects/dev/lettreai/nb/audio/templates'
templ = JinManager(templ_dir)
scheme1="transcribe_vorlesung2"
scheme2="quartohead_var1"
params = templ.schema[scheme1]['params']
print(f"para={params}")

para=['TRANSCRIPT']


In [8]:
models = ["gpt-4o", "claude-opus","claude-haiku","llama3","llama3-70","dbrx","groq-llama3","groq-mixtral"]
themen=["ai_Vorl1","ai_Vorl2","ai_Vorl3","ai_Vorl4"]
model_select = models[1]
n = 4  # 
for thema in themen[3:4]:
    result = ""
    print(thema)
    transkript=ai_vorl[thema]
    chsize = len(transkript)//n
    chunks = [transkript[chsize*i:chsize*(i+1)] for i in range(0, n)]
    for chunk in chunks:
        par = {"TRANSCRIPT": chunk}
        instruct = templ.render(scheme1, par)
        head=templ.render(scheme2, {'SUBTITLE':thema})
        gg, vendor = instruct_execute(sys_mess,instruct, model_select)
        display(Markdown(gg)) if isinstance(gg, str) else display(gg)
        result += gg
    with open(f"{thema}_{model_select}.qmd", "w") as file: file.write(head+result)

ai_Vorl4
Chosen model: claude-3-opus-20240229, Vendor: Anthropic


# Begrüßung und Einführung

Guten Tag, meine Damen und Herren! Ich begrüße Sie herzlich zu unserer heutigen Vorlesung "Philosophie der AI". Leider ist das Touchpanel hier im Hörsaal seit Montag defekt und die Uni hat es bis heute nicht geschafft, das Problem zu beheben. Daher funktioniert zwar die Projektion, aber die Mikrofone nicht. Ich werde versuchen, so laut wie möglich zu sprechen. Falls Sie mich nicht gut genug verstehen können, melden Sie sich bitte umgehend, damit ich die wichtigen Punkte nochmals wiederholen kann.

# Aktuelle Entwicklungen in der KI

Wie Sie sicherlich mitbekommen haben, wenn Sie das Geschehen rund um die Künstliche Intelligenz in den Medien verfolgen, kommen derzeit jede Woche neue Modelle auf den Markt, die mit immer größeren Versprechungen verbunden sind. Es entsteht fast der Eindruck, als seien schon alle Probleme gelöst oder zumindest kurz davor, gelöst zu werden. Diese Hyperaktivität zeigt sich bei verschiedensten Firmen. Mittlerweile sind alle großen Computer-Technologie-Konzerne an der Entwicklung von KI-Modellen beteiligt.

Erst letzte Woche wurden von OpenAI, Anthropic, Google und anderen Unternehmen diverse neue Modelle vorgestellt, deren Leistungsfähigkeit anhand unterschiedlicher Bewertungsskalen gemessen wird. Wir haben diese Skalen bereits in der letzten Vorlesung diskutiert. Ähnlich wie beim Schachspiel, wo die Spielstärke mit der Elo-Zahl angegeben wird, gibt es auch für KI-Modelle vergleichbare Messlatten. Ich werde nicht im Detail auf die einzelnen Skalen eingehen, aber Fakt ist, dass die Bewertungen der Leistungsfähigkeit stark von den angelegten Maßstäben abhängen.

Derzeit sind die Leistungsmaßstäbe noch so gering, dass alle Modelle die gestellten Tests mit Bravour meistern und sich entsprechend gut und leistungsfähig in der Öffentlichkeit präsentieren können. Allerdings erfüllen sie bei Weitem noch nicht alle Anforderungen, die wir möglicherweise an eine echte Künstliche Intelligenz stellen würden.

# Erwartungen an KI-Modelle 

Ich möchte heute mit Ihnen über unsere Erwartungen an KI-Modelle sprechen und Ihnen ein eigenes Projekt vorstellen, an dem Sie auch gerne mitarbeiten können. In der zweiten Hälfte der Vorlesung werde ich die Bedingungen und Anforderungen für diese Mitentwicklung eines neuen KI-Modells näher erläutern. Sie sollen dann klar verstehen, was von Ihnen erwartet wird und welche Leistungsnachweise Sie erbringen können.

## Generative KI und KI-Charaktere

In den bisherigen Vorlesungen haben wir KI-Modelle als generative KI eingeführt. Technisch gesehen bedeutet das, dass ein Input, beispielsweise ein Text, eine Chat-Nachricht oder eine Interaktion über Audio, Video oder andere Medien, eine Eingabe definiert. Aus diesem Input generiert das Modell dann einen Output. Dieser generierte Output ist das Leistungsergebnis, der Service oder die Funktionalität, die von den KI-Modellen produziert wird. Daher spricht man auch von generativer KI.

In der letzten Vorlesung habe ich bereits den Begriff des KI-Charakters eingeführt. Damit ist gemeint, dass wir die Art und Weise, wie generative Modelle reagieren, mitgestalten können. Das geht so weit, dass man beispielsweise festlegen kann, in welcher Sprache eine Antwort gegeben werden soll. 

Die sprachlichen Fähigkeiten der aktuellen Modelle sind mittlerweile so gut, dass kaum noch jemand bezweifelt, dass die Ausgaben auch für sprachkompetente Muttersprachler fast fehlerfrei sind. Diese Leistung ist in der Tat beeindruckend, was sich besonders gut an den Übersetzungsfähigkeiten der KI-Modelle zeigen lässt. Die Modelle geben nicht nur irgendwelche Texte aus, sondern bedeutungsvolle Texte, die in einer anderen Sprache neu formuliert werden, ohne dass diese Übersetzungen zuvor irgendwo publiziert wurden. Das ist eine der herausragenden Qualitäten dieser Modelle.

## KI - Ein Marketingbegriff?

Ein KI-Charakter ist also so etwas wie eine Individualität oder ein besonderes Reaktionsvermögen eines Modells. Ich möchte diesen Gedanken heute weiterentwickeln und mit Ihnen darüber reflektieren, was wir eigentlich unter einem KI-Modell verstehen können und wollen. Dabei geht es nicht nur darum, was uns von anderen als "das ist jetzt KI" vorgesetzt wird. Denn das ist zunächst einmal nichts anderes als ein Marketingbegriff.

Wie ich bereits erwähnt habe, wurde der Begriff der Künstlichen Intelligenz in den 1950er Jahren unter anderem von Herbert Simon und zwei anderen Kollegen geprägt, um eine potenzielle, visionäre Maschine oder Technologie zu charakterisieren, die die kognitive Leistungsfähigkeit von Menschen imitieren sollte. Lange Zeit wurde dieses Ziel nicht erreicht, aber jetzt sind wir an dem Punkt angelangt, wo wir diese Leistungsfähigkeit auf gleichem Niveau erreichen können. Das nennt man dann KI.

# Anforderungen an zukünftige KI

Ich möchte heute die Perspektive umkehren. Anstatt uns nur vorsetzen zu lassen, was als KI bezeichnet wird, und dann als Forscher herauszufinden, was uns da eigentlich präsentiert wird, wollen wir uns überlegen, was wir selbst von einer KI erwarten, die vielleicht noch entwickelt werden muss, aber in absehbarer Zeit auch entwickelt werden wird. Dann werden wir sehen, dass das, was uns heute begegnet, bei Weitem nicht diese Anforderungen erfüllt.

In den vielen Präsentationen, die wir fast täglich von unterschiedlichen Firmen sehen, werden zwar beeindruckende Leistungen vorgeführt, wie beispielsweise die Simultanübersetzung, die vorgestern von OpenAI mit dem neuen GPT-4.0-Modell eindrucksvoll demonstriert wurde. Aber das sind nur Teilaspekte dessen, was wir eigentlich von KI erwarten. Die fehlenden Teile werden in all diesen Präsentationen natürlich nicht erwähnt oder unterschlagen. Dabei sind sie extrem wichtig für das, was wir von KI-Modellen eigentlich erwarten sollten.

## Defizite aktueller KI-Modelle

In der zweiten Vorlesung haben wir bereits einige Defizitbereiche kennengelernt. Einer der am häufigsten diskutierten ist die sogenannte Halluzination. Das bedeutet, dass die KI-Modelle zwar wunderbar formulieren können, aber den Wahrheitsgehalt ihrer Aussagen nicht validieren oder rechtfertigen können. Diese Fähigkeit fehlt ihnen.

Auch der Begriff des Wissens wird in diesem Zusammenhang oft inflationär benutzt, ohne dass sich die Akteure darüber im Klaren sind, was es eigentlich heißt, über Wissen zu verfügen. Besonders Informatiker neigen dazu, den Begriff der Information mit dem des Wissens zu verwechseln. Dabei sind das zwei völlig verschiedene Dinge.

Der Begriff der Information ist in der Mathematik seit Langem definiert, beispielsweise durch die Informationstheorie von Shannon. Dieser Begriff der Information und des Informationsgehalts hat jedoch nichts mit Wissen zu tun. Wenn eine Firma wie Google davon spricht, Wissensbäume, Wissensrelationen oder Wissensnetzwerke zu erstellen, ist das nichts anderes als Propaganda. Die verwendete Terminologie wird den epistemischen oder philosophischen Ansprüchen nicht gerecht und täuscht Leistungsfähigkeiten vor, die nicht vorhanden sind.

Dennoch sind beeindruckende Kompetenzen vorhanden, die wir bereits kennengelernt haben. Das, worüber wir jetzt sprechen und was aktuell so intensiv diskutiert wird, sind die sogenannten Large Language Models (LLM). Diese Modelle haben die Fähigkeit, als Reaktion auf eine Eingabesequenz von symbolhaltigen Inhalten, einschließlich Videobildern und ähnlichen Daten, inhaltlich korrespondierende Ausgaben zu generieren.

## Unterschied zwischen Information und Wissen

Sie fragten, wie sich der Begriff des Wissens von dem der Information unterscheidet. Ich werde diese Frage in der Vorlesung noch ausführlicher behandeln, aber lassen Sie mich zunächst die klassisch-philosophische Definition von Wissen anführen, die meiner Meinung nach allerdings auch nicht ausreicht.

Die sogenannte platonische Definition besagt, dass Wissen aus drei Komponenten besteht: Es muss eine wahre, gerechtfertigte Meinung von Akteuren sein. Keine dieser drei Bedingungen ist bei bloßen Informationen erfüllt. Information ist eher so etwas wie das, was sich vom Rauschen abhebt - eine Signalhaftigkeit. Das Ausmaß der Signalhaftigkeit ist die Information. 

Schon an dieser vagen Definition erkennt man, dass hier weder von Inhalten, von Wahrheit, von Rechtfertigung noch von einer Meinung die Rede ist. Information und Wissen sind völlig unterschiedliche Kategorien und es ist ein großer Fehler, diese Begriffe zu vermischen.

Ich halte es für extrem wichtig, dass KI-Modelle in Zukunft als Wissensakteure auftreten können. Das ist etwas, was wir zu Recht von einer Entität erwarten, die sinnvollerweise als intelligent bezeichnet werden soll. Die aktuellen Modelle haben diese Fähigkeit nicht, was sich unter anderem an dem angesprochenen Problem der Halluzination zeigt.

# Herausforderungen für die Entwicklung zukünftiger KI

Man versucht dieses Problem dadurch zu lösen, dass die Menge der Textgrundlagen, die für das Training verwendet werden, immer weiter vergrößert wird. Aktuelle, bedarfsgerechte Quellen, die spezifische Informationen liefern, werden in die Antwortgenerierung mit einbezogen. Das ist eine Technik, um das Problem anzugehen, aber sie erfüllt bei Weitem nicht die Anforderungen, die wir insgesamt an Wissen stellen. Die Modelle reproduzieren Literaturangaben, qualifizieren sich aber nicht als Vermittler von echtem Wissen zu einem Thema. Darauf werden wir noch näher eingehen.

Meine Ausführungen sollen aber nicht negativ verstanden werden in dem Sinne, dass ich hier mit dem Finger auf Defizite und Schwächen der aktuellen Modelle zeige, um diese abzuwerten. Im Gegenteil, ich sehe das durchaus positiv. Es ist eine interessante, auch philosophische Herausforderung, genau die Kriterien zu definieren, die wir als Leistungsansprüche für die Entwicklung zukünftiger KI formulieren sollten. Das sind die Maßstäbe, an denen die Qualität dieser Modelle gemessen werden sollte.

Das bezieht sich auf meine anfängliche Kritik an den derzeitigen Bewertungsmaßstäben für die Leistungsfähigkeit der Modelle. Wenn diese Modelle einige Eingangsexamen lösen können, ist das auf dem Niveau von Einsteigertests vielleicht halbwegs beeindruckend, aber mehr auch nicht. Das Kompetenzniveau, das Sie beispielsweise nach einem Bachelorabschluss erreichen, ist noch lange nicht in Reichweite dieser Modelle.  

## Sprachkompetenz als Kernkompetenz aktueller KI-Modelle

Was wir an Kompetenzen derzeit haben, sind also Large Language Models (LLM). Das bedeutet, dass diese Modelle erstaunlicherweise eines schon sehr gut beherrschen, nämlich Sprache zu verwenden und zu verarbeiten. Das ist eine enorme Errungenschaft, aber eben auch nicht mehr.

Beim Erwerb dieser Sprachkompetenz wird natürlich nicht nur die grammatische Kompetenz vermittelt, sondern anhand von Milliarden von Beispielen werden diese Sprachkompetenzen auch inhaltlich repräsentiert. Bei jeder Antwort, die mit diesen trainierten Beispielen generiert wird, entsteht der Eindruck, dass auch ein sinnvoller Inhalt produziert wird. Und das ist das tiefere Geheimnis der Halluzination.

Die Ergebnisse scheinen deshalb so plausibel zu sein, weil es irgendwo in dem gigantischen Trainingskorpus eine oder mehrere Formulierungen gegeben hat, die sich zu einer scheinbar schlüssigen Antwort zusammensetzen lassen. Deren Wahrheitsgehalt wurde jedoch nie überprüft. Sprachkompetenz ist also vorhanden und lässt sich auch prüfen. Wir können inzwischen leicht Übersetzungen erstellen lassen und selbst bewerten, ob sie gelungen sind oder nicht. 

Wir haben auch gesehen, dass die Sprachmodelle durch einfache Systeme um Zusatzkompetenzen erweitert werden können. In der Terminologie der KI-Ingenieure spricht man hier vom Kontext.

# Kontexte und Handlungsanweisungen in der Interaktion mit KI

Der Kontext besteht zunächst einmal aus all den zusätzlichen Textinformationen, die zu einer spezifischen Instruktion als Input für das Sprachmodell hinzugegeben werden müssen, um einen gewünschten Output zu generieren. Auf der Nutzerseite gibt es erstaunlicherweise nicht viel mehr. Die eigentliche Konstruktion ist also recht einfach, auch wenn im Hintergrund technisch natürlich enorm viel passiert. 

Die Art und Weise, wie die Informationen verarbeitet werden und wie die verschiedenen antrainierten Kompetenzebenen miteinander verschränkt sind, ist durchaus beeindruckend. Auf diese technischen Aspekte will ich aber nicht näher eingehen. Ich möchte die Interaktion mit KI-Modellen so behandeln, als würden wir mit einer Person mit bestimmten Kompetenzen sprechen, ohne uns Gedanken über die neuronale Struktur ihres Gehirns zu machen. So ähnlich sollten wir meiner Meinung nach auch mit KI-Modellen umgehen.

## Instruktionen als Handlungsanweisungen

Der Kontext ist also wichtig für die antrainierten Informationsmengen, die für eine spezifische Anfrage einen gewünschten Output generieren. Innerhalb dieser Eingabeinformationen gibt es oft eine Formulierung oder einen Text, den man als Instruktion verstehen kann. Das kann beispielsweise eine Aufforderung sein wie "Übersetze diesen Text". Der zu übersetzende Text wird dann als Kontext mit eingegeben.

Wenn dieser Text entsprechend formuliert ist, wird er

Chosen model: claude-3-opus-20240229, Vendor: Anthropic


# Belief-Desire-Modell einer Handlung

In der Philosophie und insbesondere in der Handlungstheorie spielt das sogenannte Belief-Desire-Modell eine zentrale Rolle. Dieses Modell besagt, dass für die Ausführung einer Handlung zwei grundlegende Elemente vorhanden sein müssen: zum einen eine Vorstellung oder ein Ziel, das erreicht werden soll (Desire), und zum anderen eine Überzeugung über die vorliegenden Situationsgegebenheiten, also eine faktische Beschreibung der Situation, aufgrund derer etwas getan werden muss (Belief). Diese beiden Elemente sind logisch gesehen völlig unterschiedlich, bilden aber zusammen das, was in der englischsprachigen philosophischen Literatur als "Belief-Desire-Duett" bezeichnet wird.

Die Handlungstheorie hat sich eingehend mit den komplexen Netzwerken von Belief-Desire-Verbindungen befasst, und zwar nicht nur für Individuen, sondern auch für große Kollektive. Interessanterweise werden diese Netzwerke derzeit von keinem der existierenden KI-Modelle auch nur ansatzweise realisiert, was das enorme Entwicklungspotenzial in diesem Bereich verdeutlicht.

## Instruktionen als Kern der KI-Modelle

Wenn wir uns die aktuellen KI-Modelle genauer ansehen, stellen wir fest, dass sie im Wesentlichen Instruktionen ausführen. Diese Instruktionen lassen sich in natürlicher Sprache durch Handlungsanweisungen ausdrücken und beschreiben, welche Handlung unter welchen Zielen und mit welchen Mitteln ausgeführt werden soll. Dieses Prinzip lässt sich bis hin zur Analyse wissenschaftlicher Texte nachvollziehen: In der Wissenschaftskommunikation werden durch Publikationen konkrete Ausführungen wissenschaftlicher Handlungsoperationen kommuniziert, die von den Rezipienten aufgenommen und weitergesponnen werden.

Es ist eine richtige Entwicklung, dass sich die KI-Forschung mittlerweile auf die Umsetzung von Instruktionen konzentriert. Die ursprüngliche Idee, das Interface zwischen Mensch und Maschine durch eine dialogische Gesprächssituation, wie beispielsweise in Chatbots, zu kanalisieren, verschiebt sich hin zu einer zwar ebenfalls dialogisch geführten Interaktion, bei der es aber im Kern um Instruktionen und Handlungsanweisungen geht.

# Lernen von Kompetenz im Hintergrund

Was wir als Nutzer dieser KI-Modelle oft nicht sehen, ist das kontinuierliche Lernen von Kompetenz im Hintergrund. Man könnte den Eindruck gewinnen, dass diese Modelle bereits eine vollständige Kompetenz mitbringen und diese nur noch anwenden und dem Nutzer zur Verfügung stellen. Doch das ist nicht der Fall.

## Interaktives Lernen durch Nutzerfeedback

Bereits beim Chatten findet ein interaktiver Vorgang statt, bei dem die Reaktionen, Korrekturen und Rückmeldungen des Nutzers in die Konstruktion eines geeigneten Kontexts einfließen. Dieser Kontext ist entscheidend, um die Instruktionen so zu führen, dass am Ende ein Ergebnis steht, das für den Nutzer einen Wert hat und seinen Erwartungen entspricht.

Die KI-Modelle leben davon, dass die Nutzer Interaktionen und Informationen einbringen, die in den jeweiligen Funktions- und Kompetenzbereich des Modells integriert werden. Im Hintergrund ist all dies implementiert, sodass die Modelle aus den Reaktionen der Nutzer ständig lernen können. Die rasante Abfolge von Versionserneuerungen dieser Modelle ist nicht nur Ausdruck der technischen Weiterentwicklung, sondern auch des stetigen Verbesserns durch die millionenfache Interaktion mit den Nutzern.

### Beispiel: Biografische Informationen zu Leonhard Euler

Ein konkretes Beispiel, das ich selbst ausgetestet habe, betrifft biografische Informationen zum Mathematiker Leonhard Euler. Die Frage, wer Eulers zweite Ehefrau war, ist historisch nicht ganz einfach zu beantworten. Zu Beginn gaben die KI-Modelle auf diese Frage die skurrilsten Antworten, regelrechte Halluzinationen. Doch nachdem ich die Anfrage zehnmal beim gleichen Modell gestellt hatte, wusste es am Abend die richtige Antwort. Der Grund dafür ist, dass die Modelle so aufgebaut sind, dass sie Korrekturen und Rückmeldungen der Nutzer aufzeichnen und in ihre Wissensbasis integrieren.

## Lernen durch Nutzerdaten und externe Quellen

Das Lernen von Kompetenzen gehört also untrennbar zu diesen KI-Modellen dazu, auch wenn es derzeit noch auf einem relativ niedrigen Niveau stattfindet und sich im Wesentlichen auf die Verarbeitung von Nutzerreaktionen beschränkt. Doch das Potenzial reicht noch viel weiter.

Die Anbieter der Modelle offerieren den Nutzern beispielsweise die Möglichkeit, eigene PDFs hochzuladen, um die darin enthaltenen Informationen für die Beantwortung von Anfragen nutzbar zu machen. Das bringt zwar einen Mehrwert für den einzelnen Nutzer, dient aber gleichzeitig als Qualitätsindikator für die Firmen, um zu erkennen, welche Informationen für die zukünftige Verbesserung der Modelle relevant sein könnten. Diese Informationen fließen dann in den stetigen Lernprozess der Modelle ein, einschließlich des Wissenshintergrunds.

### Googles Digitalisierungsprojekt mit Bibliotheken

Ein bemerkenswertes Beispiel für den Aufbau eines solchen Wissenshintergrunds ist Googles Projekt zur Digitalisierung von historischen, urheberrechtsfreien Beständen großer Bibliotheken weltweit. Mit enormem technischem und finanziellem Aufwand wurden und werden in Digitalisierungsstationen, unter anderem in Berlin und München, wertvolle Altbestände digitalisiert und gesichert. Dieses Projekt hat einen immensen Wert für die Erhaltung unseres kulturellen Erbes.

Doch warum hat Google diesen Aufwand betrieben? Heute wird deutlich, welchen Wert diese digitalisierten Bestände haben: Sie bilden einen umfassenden Informationshintergrund, der für das Training von KI-Modellen genutzt werden kann. Die eigentliche Aufbereitung und Verarbeitung dieser Inhalte hat noch kaum begonnen, doch in den nächsten Jahren wird mit Sicherheit eine intensive Auseinandersetzung mit diesem digitalisierten Kulturwissen erfolgen.

# Generierung und Kontext in der Interaktion mit Chatmodellen

In der letzten Vorlesung haben wir anhand einiger Beispiele diskutiert, wie die Interaktion mit einem Chatmodell gestaltet werden kann. Dabei haben wir gesehen, dass die Modelle den Kontext der Anfrage berücksichtigen und entsprechend präzisere Antworten geben können.

## Kontextbezogene Antworten

Wenn wir beispielsweise nach Johann Wolfgang Goethe fragen, präferiert das Modell aufgrund des Kontextes eine bestimmte Person, auch wenn es theoretisch mehrere Personen mit diesem Namen geben könnte. Stellen wir in der Folge eine Frage wie "Wo lebte er die meiste Zeit?", kann das Modell diese Frage korrekt beantworten, indem es den Bezug zu der zuvor genannten Person herstellt. Das funktioniert, weil die Modelle in der Lage sind, den deiktischen Ausdruck "er" richtig zuzuordnen und den Kontext zu berücksichtigen.

## Metaebene der Instruktionen

Darüber hinaus haben wir gesehen, dass wir den Modellen Instruktionen geben können, die sich nicht nur auf die Klärung einer Sachfrage beziehen, sondern auch auf einer Metaebene angesiedelt sind. So konnten wir beispielsweise die Anweisung geben, nur die gestellten Fragen zu beantworten und keine zusätzlichen Ausführungen zu machen. Anfangs war das Programm so eingestellt, dass es möglichst viele Informationen zu Goethe und seinen Zeitgenossen ausgab, um zu beeindrucken. Durch die Metainstruktion konnten wir jedoch eine Beschränkung auf die wesentlichen Aspekte erreichen.

## Vielschichtigkeit der Kompetenzbereiche

Die Modelle, von denen hier die Rede ist, realisieren also unterschiedliche Aspekte der Nutzungsweise und der Informationsverarbeitung, die durch die natürliche Umgangssprache formuliert und eingegeben werden können. Diese verschiedenen Ebenen sind extrem vielschichtig, und die Leistungsfähigkeit der Modelle liegt im Wesentlichen in der Komposition dieser jeweiligen Kompetenzsektoren oder -felder.

# Grenzen aktueller KI-Modelle

Anhand des Beispiels der Frage nach dem Briefwechsel zwischen Goethe und Friedrich II. haben wir gesehen, dass es durchaus einfache Fragen gibt, die von den aktuellen Modellen nicht zufriedenstellend beantwortet werden können. Das liegt daran, dass den Modellen die nötige Evidenz fehlt, um eine fundierte Aussage treffen zu können.

- Lagen dem Modell überhaupt Informationen über die Gesamtkorrespondenz vor? 
- Wenn kein Brief an Friedrich II. dokumentiert ist, was lässt sich daraus schließen? 
- Haben die beiden tatsächlich nicht miteinander kommuniziert, obwohl es zeitlich möglich gewesen wäre?

Solche Fragen lassen sich durch die aktuellen KI-Modelle nicht beantworten. Ob sie grundsätzlich lösbar sind, ist eine andere Frage, und ich hoffe, dass in dieser Vorlesung deutlich wird, wie scheinbar unlösbare Probleme für KI-Modelle doch lösbar werden könnten.

# Perspektivwechsel: Erwartungen an eine philosophische KI

Lassen Sie uns nun einen Perspektivwechsel vornehmen und überlegen, was wir von einem KI-Charakter erwarten würden, der die bisher diskutierten Eigenschaften und Kompetenzen beherrscht und umsetzt. Dabei möchte ich mich auf das Grundmodell der derzeit verfügbaren Technologien konzentrieren, nämlich auf instruktionsausführende technische Akteure oder Agenten.

## Allgemeine künstliche Intelligenz (AGI)

In diesem Zusammenhang stellt sich die Frage nach der sogenannten allgemeinen künstlichen Intelligenz oder AGI, die derzeit häufig diskutiert wird. Unternehmen wie OpenAI haben es sich zum Ziel gesetzt, möglichst schnell eine generelle künstliche Intelligenzkompetenz zu entwickeln. Ich habe bereits vor einigen Vorlesungen meine Zweifel geäußert, ob dieses Ziel überhaupt wünschenswert ist. Letztlich wird sich diese Frage durch die technologische Entwicklung von selbst beantworten.

## Fokus auf spezifische Kompetenzbereiche

Meiner Meinung nach sollten wir von einer philosophischen KI nicht erwarten, dass sie ein universell kompetentes Genie ist, das alles Wissen seiner Zeit beherrscht. Solche Zuschreibungen, wie sie beispielsweise Leibniz oder anderen historischen Figuren gemacht wurden, halte ich ohnehin eher für Rückprojektionen als für historische Tatsachen.

Stattdessen sollten wir uns auf bestimmte Kompetenzsektoren konzentrieren, die für eine philosophische KI relevant und notwendig sind. Wie wir am Beispiel des Briefwechsels zwischen Goethe und Friedrich II. gesehen haben, gibt es viele scheinbar einfache Fragen, die von den aktuellen Modellen nicht zufriedenstellend beantwortet werden können. Die dafür erforderlichen Kompetenzen gehen über das hinaus, was derzeit möglich ist, und erfordern weitere Fähigkeiten.

## Semantische Suche

Ein Kompetenzbereich, den wir bereits in unsere Liste aufgenommen haben, ist die semantische Suche. Im Gegensatz zur reinen Textsuche, wie sie beispielsweise von Google angeboten wird, geht es hier um die Suche nach Inhalten und nicht nur nach bestimmten Formulierungen. Wenn wir nach den besten Rezepten für ein bestimmtes Gericht suchen, sind wir im Grunde an den Inhalten interessiert, nicht an den spezifischen Bezeichnungen der Zutaten. Die aktuellen KI-Modelle sind in der Lage, solche semantischen Suchen durchzuführen.

## Reasoning

Ein weiterer Bereich, der noch am Anfang steht, aber von großer Bedeutung ist, ist das Reasoning. Damit ist das Schlussfolgern im weitesten Sinne gemeint, also nicht nur im Sinne der Logik oder Mathematik. Das menschliche Schlussfolgern ist viel umfassender und bezieht sich auf alle möglichen Bereiche des Nachdenkens mit einem bestimmten Ergebnis.

Reasoning umfasst einerseits das Nachdenken, um etwas vorherzusagen oder abzuleiten, also eine Instruktion in eine Vorhersage umzusetzen. Andererseits geht es auch darum, zu begründen, warum etwas geeignet ist oder warum etwas der Fall ist. Diese beiden Hauptbewertungskategorien des Reasonings sind mit den derzeitigen KI-Modellen praktisch nicht zu realisieren.

Chosen model: claude-3-opus-20240229, Vendor: Anthropic


# Der Weg zu verantwortungsbewussten KI-Modellen

Meine sehr verehrten Damen und Herren,

stellen Sie sich vor, wir könnten in Zukunft nicht nur Informationen aus KI-Modellen abrufen, sondern auch genau nachvollziehen, aus welchen Quellen diese Informationen stammen. Eine solche Fähigkeit wäre zwar ein Triumph, aber noch lange keine Rechtfertigung für die Ausgaben der Modelle. Denn dafür braucht es mehr als nur einen Quellennachweis - es braucht eine fundierte Begründung und ein umfassendes Reasoning.

## Die Notwendigkeit der historischen Kontextualisierung

Ein entscheidender Aspekt, der bei jeder Anfrage an KI-Modelle berücksichtigt werden muss, ist die historische Kontextualisierung. Nehmen wir das Beispiel von Goethe und Friedrich II. Bei einer Anfrage zu diesen beiden Persönlichkeiten geht es nicht nur um spezifische Informationen über sie selbst, sondern auch um ihr komplexes Kommunikations- und Akteursnetzwerk. All diese Informationen müssen identifiziert, hinzugezogen und maschinell ausgewertet werden - eine Aufgabe, die derzeit noch manuell erledigt werden muss, da die historische Kontextualisierung in den Modellen fehlt.

## Die Herausforderung der Beurteilung historischer Hypothesen

Ein Spezialfall, der die aktuellen Fähigkeiten von KI-Modellen übersteigt, ist die Beurteilung historischer Hypothesen anhand von Referenzen und Evidenzen. Doch genau diese Kompetenz wird in Zukunft von entscheidender Bedeutung sein.

## Die Bedeutung epistemischer Kompetenz

Um wirklich verantwortungsbewusst zu agieren, müssen KI-Modelle in der Lage sein, ihre Wissensansprüche zu rechtfertigen, Quellen auszuweisen, mit Kritik umzugehen, Widerlegungen durchzuführen und zu verstehen, wie offene Fragen durch weitere Forschung einer endgültigen Beurteilung zugeführt werden können. All diese Fähigkeiten fallen in den Bereich der epistemischen Kompetenz - ein Bereich, in dem die derzeitigen Modelle noch große Defizite aufweisen.

# Die Idee der Individualität von KI-Modellen

Lassen Sie uns einen Gedanken wagen, der auf den ersten Blick abwegig erscheinen mag: Die Idee, KI-Modellen eine gewisse Individualität zuzuschreiben. In der Diskussion um die ethische Verantwortbarkeit von KI gehen wir bisher von zwei Grundprämissen aus:

1. Die Frage der Verantwortung, etwa in der Anwendung von Modellen in verschiedenen Bereichen wie der juristischen Beurteilung.
2. Die Annahme, dass Maschinen keine rechtsfähigen Objekte oder Subjekte sind und somit nicht haftbar gemacht werden können.

Doch was, wenn wir diese Prämissen hinterfragen? Was, wenn wir KI-Modellen eine Individualität zusprechen, die sie zu rechtsfähigen Körperschaften macht - ähnlich wie Firmen oder Universitäten?

## Die Voraussetzungen für eine KI-Körperschaft

Damit ein KI-Modell als Körperschaft anerkannt werden kann, müsste es einige Voraussetzungen erfüllen:

- Persistenz und Dauerhaftigkeit
- Die Fähigkeit, für Konsequenzen zur Verantwortung gezogen zu werden
- Eine eigene Individualität, die nicht an ein bestimmtes Unternehmen gebunden ist

Wenn diese Bedingungen erfüllt sind, könnte eine solche KI-Körperschaft möglicherweise viele der Probleme lösen, die sich derzeit in der Technologiefolgenabschätzung stellen und für die die aktuellen Haftungskonstruktionen unzureichend sind.

# Historische Vorbilder für die Gestaltung von KI-Modellen

Die Idee, KI-Modelle nach bestimmten Vorbildern zu gestalten, ist keineswegs neu. Lassen Sie uns einen Blick in die Geschichte werfen und drei bedeutende Metaphern betrachten.

## Der vitruvianische Mensch - Die Proportion als Naturgesetz

Der vitruvianische Mensch, wie er von Leonardo da Vinci dargestellt wurde, steht für die Idee, dass die einzelnen Bestandteile eines Ganzen in der richtigen Proportion zueinander stehen müssen, um ein funktionierendes Ganzes zu bilden. Übertragen auf KI-Modelle bedeutet dies, dass die einzelnen Kompetenzen in ein ausgewogenes Verhältnis gesetzt werden müssen, um ein verantwortliches und individuiertes Modell zu schaffen.

## David - Die Freiheit des selbstbestimmten Lebens

Michelangelos David symbolisiert die Freiheit des selbstbestimmten Lebens und die Eigenverantwortlichkeit des Individuums. In Bezug auf KI-Modelle mahnt uns diese Metapher, darauf zu achten, dass ihre Anwendung nicht zu einem Überwachungsstaat führt, der die individuellen Freiheiten beschränkt.

## Der Künstler als Erklärer - Die Fähigkeit zur Rechtfertigung

Joseph Beuys' Performance, in der er als Künstler mit goldener Maske einem toten Hasen eine Ausstellung erklärt, steht für die Fähigkeit, Thesen, Inhalte und Ergebnisse rechtfertigen und erklären zu können. Dies ist eine entscheidende Anforderung an KI-Modelle: Sie müssen in der Lage sein, ihre Ausgaben zu begründen und epistemisch zu rechtfertigen.

# Das Projekt "Magister AI Faustus"

Unter dem Arbeitstitel "Magister AI Faustus" möchte ich Sie einladen, sich in dieser Vorlesung und darüber hinaus mit den Fragen der Gestaltung verantwortungsbewusster KI-Modelle auseinanderzusetzen. Ob in Bachelorarbeiten, Leistungsnachweisen oder Masterarbeiten - lassen Sie uns gemeinsam erforschen, wie wir KI-Modelle schaffen können, die nicht nur leistungsfähig, sondern auch ethisch vertretbar und epistemisch fundiert sind.

Der Titel "Magister AI Faustus" ist dabei eine Anspielung auf die lange historische Tradition des Faust-Stoffes, der nicht erst mit Goethe begann, sondern bereits in Werken wie Christopher Marlowes "The Tragical History of the Life and Death of Dr. Faustus" behandelt wurde. Lassen Sie uns in diesem Sinne zu modernen Fausten werden - nicht im Streben nach grenzenlosem Wissen um jeden Preis, sondern im verantwortungsvollen Gestalten einer KI, die dem Menschen dient und ihn nicht beherrscht.

Chosen model: claude-3-opus-20240229, Vendor: Anthropic


# Die Faust-Legende als Inspiration für künstliche Intelligenz

In der Faust-Legende, die von Christopher Marlowe im Jahr 1587 erstmals in Dramaform gebracht und später von Goethe in seinem berühmten Werk aufgegriffen wurde, geht es um den Gelehrten Faustus, der nach Erkenntnissen strebt und dafür sogar faustische Pakte eingeht. Diese Geschichte inspiriert uns heute, darüber nachzudenken, wie wir die Grenzen des Wissens und der Macht von KI-Modellen gestalten und den Begriff der Verantwortung in diese Modelle einfließen lassen können.

## Kooperation mit der Klassikstiftung Weimar

Um diese Ideen umzusetzen, habe ich eine Kooperation mit der Klassikstiftung Weimar abgeschlossen, der zweitgrößten Kulturstiftung Deutschlands. Die Stiftung verwaltet und präsentiert die Nachlässe von Goethe, Schiller und dem Bauhaus und wertet diese kognitiv und editorisch aus. Im Rahmen dieser Kooperation können wir für unsere Vorlesung bereits publizierte Quellen zu Goethes Biografie unter dem Link "Goethe Biographica" nutzen.

## Herausforderung an die gegenwärtigen KI-Modelle

Ich gehe davon aus, dass die gegenwärtigen KI-Modelle bestimmte Anforderungen, die sich auf Goethes Biografie beziehen, noch nicht so beantworten können, wie wir es für richtig halten. Daher sollen die Projekte in diesem Kurs kleinstmögliche Anfragen und Anforderungen formulieren, die sich auf Goethes Biografie beziehen. Ein Beispiel wäre die Frage: "Hat Goethe jemals einen Brief mit Friedrich II. gewechselt?"

## Ziel des Projekts

Das Ziel ist es, mit dem Quellbestand der Klassikstiftung Weimar aufzuzeigen, dass wir einen Kontext generieren können, der zusätzlich zur Sprachkompetenz eines KI-Modells eine Lösung für solche spezifischen Fragen ermöglicht. Es geht nicht darum, ein Riesenforschungsprojekt zu starten, sondern anhand einer einfachen Frage zu demonstrieren, wie wir mit den vorhandenen Sprachmodellen und zusätzlichen Kompetenzen eine bisher nicht lösbare Aufgabe bearbeiten können.

## Vorgehensweise

Für die Umsetzung des Projekts werde ich in der nächsten Woche eine einfache App ins Netz stellen. Dort können Sie eine Fragestellung definieren, die von keinem derzeitigen KI-Modell seriös beantwortet wird. Die Quellen sind bereits identifiziert, sodass keine aufwendige Datenbanksuche nötig ist. Ihre Aufgabe wird es sein, mit den Sprachmodellen in natürlicher Sprache eine Kompetenz zu formulieren, die von dem Modell berücksichtigt wird.

Diese zusätzlichen Informationen definieren Kompetenzen, die derzeit in keinem der vorliegenden KI-Modelle enthalten sind, aber Sektoren von Kompetenzfeldern identifizieren, die wir zukünftig erwarten. Es sind neue Instruktionen für eine gebildete oder "lettere" KI, die diese Zusatzkompetenzen haben soll.

# Die biografischen Quellen zu Goethes Leben

Die Klassikstiftung Weimar stellt uns umfangreiche Materialien zur Verfügung, die sich auf das Leben von Goethe beziehen:

- Tagebücher zwischen den Jahren 1775 bis 1787 (12 Jahre)
- 15.000 überlieferte Briefe von Goethe an mehr als 1.400 Adressaten
- 20.000 überlieferte Briefe an Goethe von circa 3.800 Adressaten
- 40.000 dokumentierte Zeugnisse aus und zum Leben von Goethe jenseits von Briefen und eigenen Tagebüchern (sogenannte "Begegnungen und Gespräche")

Diese Zahlen sollen nicht die Heldenhaftigkeit Goethes unterstreichen, sondern zeigen, mit welcher Vielfalt an Materialien man sich auseinandersetzen muss, wenn man sich der Biografie einer Person wie Goethe nähert. Neben der schieren Menge ist auch die Verschiedenartigkeit der Dokumente beachtlich - Tagebücher, Briefe von und an Goethe sowie Zeugnisse unterschiedlichster Art.

## Die Herausforderung der Kontextualisierung

Um eine scheinbar simple Frage wie die nach einem möglichen Briefwechsel zwischen Goethe und Friedrich II. seriös zu beantworten, muss man nicht nur diese direkten biografischen Quellen berücksichtigen, sondern auch den gesamten historischen Kontext einbeziehen. Dazu gehören Goethes Zeitgenossen und alle relevanten historischen Dokumente seiner Epoche.

Die Menge und Vielfalt des Materials, das verarbeitet werden muss, um eine solche Frage mit einem Wissensanspruch zu beantworten, ist enorm. Genau diese epistemische Kompetenz muss ein KI-Modell ebenfalls erlangen können, um seriöse Antworten geben zu können - eine Herausforderung, die derzeit noch von keinem System gemeistert wird.

## Projektaufgabe und Organisation

Wer an diesem spannenden Unterfangen mitwirken möchte, kann sich für eine Projektaufgabe kleinsten Ausmaßes melden. Schicken Sie dazu bitte eine E-Mail mit Ihrem Namen, Ihrer E-Mail-Adresse, Matrikelnummer und Ihrer Bereitschaft zur Teilnahme an mich. Die genauen Themen werden wir im Laufe der nächsten Woche aushandeln und im Juni mit der Arbeit beginnen. Ziel ist es, zum Abschluss der Vorlesung eine kleine Präsentation über die Herausforderungen an die KI der Zukunft zu geben und zu zeigen, welche kleinen Fragen derzeit noch ungelöst sind, aber mit unseren Projektergebnissen lösbar erscheinen.

Als Beispiel für eine Aufgabenstellung könnte man die erwähnten "Zeugnisse zum Leben von Goethe" heranziehen. Dabei handelt es sich um Dokumente wie Mitschriften von Gesprächen Goethes mit anderen Personen bei verschiedenen Anlässen. Für Historiker sind selbst solche scheinbar banalen Informationen wertvoll, da sie Aufschluss darüber geben, mit wem Goethe wann und wo interagiert hat. Die Herausforderung besteht darin, diese Informationen überhaupt zu finden und zu extrahieren, da sie in keiner öffentlich zugänglichen Quelle wie Wikipedia enthalten sind.

Das Webmodell für die Projektarbeiten wird von meinem Lab "Lettre AI" bereitgestellt und nächste Woche freigeschaltet. Technische Vorkenntnisse sind nicht erforderlich. Wer einen Leistungsnachweis für die Vorlesung erwerben möchte, kann die Aufgabe Anfang Juli abgeben. Bei Interesse an einer Bachelor- oder Masterarbeit zu diesem Thema können Sie mir das ebenfalls gerne signalisieren.

# Bereiche zukünftiger Kompetenzen von KI-Modellen

In den verbleibenden zwei Dritteln der Vorlesung werde ich auf folgende Bereiche eingehen, die in Zukunft zu den Kompetenzen von KI-Modellen gehören sollten:

## Textgenerierung

- Übersetzung: Wie lassen sich Übersetzungen aktiv gestalten und für jeden verständlich steuern?
- Zusammenfassung: Wie können längere Textinhalte oder -mengen inhaltlich zusammengefasst werden?
- Frage-Antwort-Dialoge: Was bringt der Dialogpartner an Informationen ein? Welche Aspekte des Charakters können wir mitgestalten (z.B. Schreibstil, Fachterminologie, Anschluss an vorherige Ausführungen)?

## Datenauswertung

- Wie werden Datenquellen ausgewertet?
- Wie werden Experten und neue Daten (z.B. aktuelle Publikationen) einbezogen?

## Ergebniskritik

- Wie geht man mit Kritik des erzielten Ergebnisses um?
- Metaregeln, evidenzbasierte Aussagen und Referenz von Nachweisen

Diese Bereiche werde ich in den kommenden Vorlesungen behandeln und anhand unseres Projekts konkretisieren. Ich freue mich über Ihr Interesse und hoffe, dass wir gemeinsam spannende Erkenntnisse gewinnen und neue Maßstäbe für die Entwicklung zukünftiger KI-Systeme setzen werden.